In [49]:
import pickle
from tqdm.notebook import tqdm
import pandas as pd
from dicts import var_vocab, args, topic_to_russian
from classes.Preprocessor import Preprocessor

In [4]:
with open("data/gpt_scores.pkl", "rb") as f:
    res = pickle.load(f)

In [6]:
best_model = None
best_score = 0
for model, score in res.items():
    if best_score < score:
        best_score = score
        best_model = model

In [8]:
print(f"{best_model} score is {best_score}")

epoch_5_num_52_loss_0.7315569369028031 score is 1179


In [13]:
df = pd.read_csv("data/df.csv", low_memory=False)
df.sample(3)

,document.id,source,stage,source_text,lemm_text,city,region,date,data,assessor,...,represent_ethicity_meaning,is_ethicity_superior_raw,is_ethicity_superior_meaning,is_ethicity_aggressor_raw,is_ethicity_aggressor_meaning,is_ethicity_dangerous_raw,is_ethicity_dangerous_meaning,comment,old_id,text_sentiment
53982,830680866,iqbuzz,3,момент упущен. Тепленькими уже не возьмёшь. Ро...,николай момент упустить тёпленький взять росси...,Москва,Москва,2015-03-27,2017-03-29 13:47:41,an_men,...,no,1.0,low,3.0,irrel,1.0,no,NaN,830680866,0.0
38909,709620378,iqbuzz,3,"Я не Шарли, я не француз, не немец. Я Магомед,...",шарль француз немец магомед чеченец алд ещё дв...,Тамбов,Тамбовская область,2015-01-17,2017-04-03 22:23:58,skuchilina,...,no,3.0,irrel,3.0,irrel,1.0,no,NaN,709620378,NaN
55582,845506835,iqbuzz,3,"так что часть русских смешаны с фино - уграми,...",ир часть русский смешать финый угр часть украи...,Балаково,Саратовская область,2015-04-04,2017-03-24 23:08:55,skuchilina,...,no,3.0,irrel,3.0,irrel,1.0,no,NaN,845506835,0.0


In [16]:
ids = df['document.id'].sample(20)

In [19]:
preprocessor = Preprocessor(df=df, args=args, var_vocab=var_vocab, topic_to_russian=topic_to_russian)

In [28]:
from torch.utils.data import DataLoader
from gpt_classes.GPTDataset import GPTDataset
from gpt_classes.GPTModel import GPTModel
import torch

In [52]:
dataset = GPTDataset(df[df['document.id'].isin(ids)], args=args, preprocessor=preprocessor)

In [53]:
loader = DataLoader(dataset, batch_size=16)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = GPTModel(device).to(device)

In [54]:
model.load_state_dict(torch.load(f"models/{best_model}", map_location=torch.device('cpu')))

<All keys matched successfully>

In [55]:
all_generated = []
all_texts = []
all_desc = []

In [57]:
with torch.no_grad():
    for i, (data, cut_data) in enumerate(tqdm(loader), 1):
        outputs = model(data, cut_data)

        for el in data:
            text, descr = el.split("Описание: ", 1)
            all_texts.append(text)
            all_desc.append(descr)

        generated = model.my_generate(cut_data)
        all_generated.extend(generated)
    

  0%|          | 0/2 [00:00<?, ?it/s]

In [62]:
for gen, text, descr in zip(all_generated, all_texts, all_desc):
    print(f"\n{gen}\n======\n")


Задание: Сгенерируй описание следующего текста, оцени тональность текста, оцени наличие этнического конфликта, наличие позитивного взаимодействия этничностей, мнение о алтаец в тексте, является ли этничность алтаец доминирующей, является ли этничность алтаец агрессором, является ли этничность алтаец опасной.
Текст: найду-сброшу. Среди тюркских народов нет случайных совпадений,чтоб род назывался одинаково у двух или трех народов,не имея при этом ни чего общего. Рода были едины и считали себя народом в древности,но по мере развала Улуса они оказались в составе разных,хотя и близких народов. Одни и те же рода и у башкир,ногаев,казахов,крымских татар. Некоторые встречаются в составе каракалпаков,узбеков,алтайцев.Даже у саха есть кипчаки.
Описание: этот текст является позитивным, в этногенезе есть позитивное взаимодействие между этичностями, этнос не является чужой, а является общим, т.к. мнение об этническом статусе 'алтаец' положительное, есть этнический конфликт,этничному народу не суще